# LLM毎に最適化されたチャット プロンプトをテンプレートから効率的に作成する

❗ **Google Colabで実行する場合、ランタイムのタイプはGPU（T4, A100, L4など）を指定すること**

**予め、[Hugging Faceのウェブサイト](https://huggingface.co/meta-llama)において、Meta Llama2およびMeta Llama3.1のライセンス許諾に同意し、モデルファイルのダウンロード許可を得る必要があります。**

## 準備

必要なPythonパッケージをインストールします。

In [ ]:
!python -m pip install --no-cache-dir --upgrade \
huggingface_hub \
transformers \
langchain \
langchain-community

Hugging Face Hubにログインします。

In [ ]:
from huggingface_hub import login
login()

## Hugging Face Transformers Templates for Chat Models

In [ ]:
from transformers import AutoTokenizer

チャットのロールと内容を定義します。

In [ ]:
chat = [
    {"role": "system", "content": "あなたは日本語ネイティブで親切なAIアシスタントです。"},
    {"role": "user", "content": "こんにちは。ご機嫌いかがですか？"},
    {"role": "assistant", "content": "とても元気です。あなたのお役にたてることがあれば何なりとお尋ねください。"},
    {"role": "user", "content": "大阪の知人へ贈る、東京の土産を提案してください。"},
]

Llama 3.1 8B Instructモデルのトーカナイザーでチャットプロンプトを生成します。

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B-Instruct")
tokenizer.use_default_system_prompt = False
prompt_string = tokenizer.apply_chat_template(chat, tokenize=False)
print(prompt_string)

Jinjaテンプレートを表示してみます。

In [ ]:
print(tokenizer.chat_template)

Llama 2 7B Chatモデルのトーカナイザーでチャットプロンプトを生成します。

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
tokenizer.use_default_system_prompt = False
prompt_string = tokenizer.apply_chat_template(chat, tokenize=False)
print(prompt_string)

## LangChain Prompt Templates

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

チャットのロールと内容をテンプレートに定義します。

In [ ]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "あなたは日本語ネイティブで親切なAIアシスタントです。"),
    ("user", "こんにちは。ご機嫌いかがですか？"),
    ("assistant", "とても元気です。あなたのお役にたてることがあれば何なりとお尋ねください。"),
    ("user", "{question}")
])

テンプレートからチャットプロンプトを生成します。

In [ ]:
prompt_value = prompt_template.invoke({"question": "大阪の知人へ贈る、東京の土産を提案してください。"})
print(prompt_value.to_string())

チャットプロンプトの内容をリストで取り出すこともできます。

In [ ]:
msg_list = prompt_value.to_messages()
print(msg_list)

## Templates for Chat ModelsをLangChainで使う

チャットのロールと内容を定義します。

In [ ]:
chat = [
    {"role": "system", "content": "あなたは日本語ネイティブで親切なAIアシスタントです。"},
    {"role": "user", "content": "こんにちは。ご機嫌いかがですか？"},
    {"role": "assistant", "content": "とても元気です。あなたのお役にたてることがあれば何なりとお尋ねください。"},
    {"role": "user", "content": "{question}"},
]

Llama 3.1 8B Instructモデルのトーカナイザーでチャットプロンプトを生成します。

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B-Instruct")
tokenizer.use_default_system_prompt = False
prompt_string = tokenizer.apply_chat_template(chat, tokenize=False)
print(prompt_string)

In [ ]:
from langchain_core.prompts import PromptTemplate

生成したプロンプトをLangChain形式のプロンプトテンプレートに変換します。

In [ ]:
prompt_template = PromptTemplate.from_template(prompt_string)
prompt_value = prompt_template.invoke({"question": "大阪の知人へ贈る、東京の土産を提案してください。"})
print(prompt_value.to_string())

## Llama 3.1 8B Instruct 4ビット量子化モデルでテスト

### llama-cpp-pythonのインストール
※実行環境により方法が異なります。以下の３通りから**該当する方法のみ**を実行してください。  
詳細は[llama-cpp-python GitHubリポジトリ](https://github.com/abetlen/llama-cpp-python)を参照してください。

#### **NVIDIA CUDA GPUの場合**

まず、nvidia-smiコマンドでCUDAバージョンを確認します。

In [ ]:
!nvidia-smi

次に、ビルド済みWheelからllama-cpp-pythonをインストールします。WheelファイルのURL末尾はCUDAバージョンにより以下のとおりです。
- `cu121`: CUDA 12.1
- `cu122`: CUDA 12.2
- `cu123`: CUDA 12.3
- `cu124`: CUDA 12.4

以下のセルで**該当するバージョンのみコメントを外し**有効にしてください。

In [ ]:
# !python -m pip install llama-cpp-python --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu121
!python -m pip install llama-cpp-python --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu122
# !python -m pip install llama-cpp-python --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu123
# !python -m pip install llama-cpp-python --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu124

NVIDIA CUDA GPU環境の場合のllama-cpp-pythonインストールは以上です。  
「LangChain Expression Language (LCEL) チェインの構築」へ進んでください。

#### **Mac (Metal) の場合**

In [ ]:
!CMAKE_ARGS="-DGGML_METAL=on" python -m pip install --no-cache-dir --upgrade llama-cpp-python

Mac (Metal) 環境の場合のllama-cpp-pythonインストールは以上です。  
「LangChain Expression Language (LCEL) チェインの構築」へ進んでください。

#### **CPUのみの場合**

In [ ]:
!python -m pip install --no-cache-dir --upgrade llama-cpp-python

CPUのみ環境の場合のllama-cpp-pythonインストールは以上です。  
「LangChain Expression Language (LCEL) チェインの構築」へ進んでください。

### LangChain Expression Language (LCEL) チェインの構築

In [ ]:
from huggingface_hub import hf_hub_download
from langchain_community.llms import LlamaCpp
from langchain_core.output_parsers import StrOutputParser

モデルをダウンロードします。

In [ ]:
model_path = hf_hub_download(
    repo_id="mmnga/Llama-3.1-8B-Instruct-gguf",
    filename="Llama-3.1-8B-Instruct-Q4_K_S.gguf"
)

モデルをロードします。

In [ ]:
llm = LlamaCpp(
    model_path=model_path,
    n_gpu_layers=-1,
    n_ctx=2048,
    f16_kv=True,
    verbose=True
)

LCELチェインを構築します。

In [ ]:
output_parser = StrOutputParser()
chain = prompt_template | llm | output_parser

### 推論

In [ ]:
for s in chain.stream({"question": "大阪の知人へ贈る、東京の土産を提案してください。"}):
    print(s, end="", flush=True)

In [ ]:
for s in chain.stream({"question": "健康的な最強の朝食メニューを考えてください。"}):
    print(s, end="", flush=True)